In [2]:
"""
Adaptive Entropy–Driven One-Time Pad (AE-OTP)

----------------------------------
Properties:
- Information-theoretic confidentiality (Shannon OTP)
- Hybrid entropy fusion (OS + chaotic map)
- One-time enforcement with secure key destruction
- Deterministic correctness, provable secrecy

Author: Research Reference Implementation
"""

import os
import secrets
import hashlib

# -------------------------------------------------
# Chaotic Entropy Generator (Logistic Map)
# -------------------------------------------------
def logistic_map_entropy(length, seed=None, r=3.999999):
    if seed is None:
        seed = secrets.randbits(64) / 2**64
    x = seed
    out = bytearray(length)
    for i in range(length):
        x = r * x * (1 - x)
        out[i] = int(x * 256) & 0xFF
    return bytes(out)

# -------------------------------------------------
# OTP Key Generation (Entropy Fusion + Extractor)
# -------------------------------------------------
def generate_otp_key(length: int) -> bytes:
    if length <= 0:
        raise ValueError("Invalid key length")
    os_entropy = os.urandom(length)
    chaotic_entropy = logistic_map_entropy(length)
    shake = hashlib.shake_256()
    shake.update(os_entropy)
    shake.update(chaotic_entropy)
    return shake.digest(length)

# -------------------------------------------------
# Core OTP Operation (XOR)
# -------------------------------------------------
def otp_xor(data: bytes, key: bytes) -> bytes:
    if len(data) != len(key):
        raise ValueError("Data and key must be equal length")
    return bytes(d ^ k for d, k in zip(data, key))

# -------------------------------------------------
# One-Time Pad Controller with Key Erasure
# -------------------------------------------------
class OneTimePad:
    def __init__(self):
        self._key = None
        self._used = False

    def load_key(self, key: bytes):
        if self._used:
            raise RuntimeError("OTP key reuse detected")
        self._key = bytearray(key)

    def encrypt(self, plaintext: bytes) -> bytes:
        if self._key is None or self._used:
            raise RuntimeError("Invalid OTP state")
        ciphertext = otp_xor(plaintext, self._key)
        self._destroy_key()
        return ciphertext

    def decrypt(self, ciphertext: bytes) -> bytes:
        return self.encrypt(ciphertext)  # XOR symmetry

    def _destroy_key(self):
        for i in range(len(self._key)):
            self._key[i] = 0
        self._used = True

# -------------------------------------------------
# Demonstration (Correctness Proof by Execution)
# -------------------------------------------------
if __name__ == "__main__":
    message = b"Perfect secrecy requires perfect discipline."
    key = generate_otp_key(len(message))

    otp = OneTimePad()
    otp.load_key(key)

    ciphertext = otp.encrypt(message)
    print("Ciphertext:", ciphertext)

    otp2 = OneTimePad()
    otp2.load_key(key)
    recovered = otp2.decrypt(ciphertext)
    print("Recovered:", recovered)


Ciphertext: b'\x1e\xf6\x8eX\xa1\xb2\x88\x84\x12\xd9\x11\xc2k\n>\xec\xaex\x146\xff\x140\xe3\x8ba>\xde\xfd\xf3\xf0\xd3\x83\xe4\x1c\xf5>T\xbe\xc8\xc8\xfd\xd5\xd9'
Recovered: b'Perfect secrecy requires perfect discipline.'


In [3]:
# ----------------------------------------
# Simple One-Time Pad (OTP) Example
# ----------------------------------------

import os

def generate_key(length):
    return os.urandom(length)  # truly random key

def otp_encrypt(message: bytes, key: bytes) -> bytes:
    return bytes(m ^ k for m, k in zip(message, key))

def otp_decrypt(ciphertext: bytes, key: bytes) -> bytes:
    return bytes(c ^ k for c, k in zip(ciphertext, key))

# -------- Example 1: Basic Text Encryption --------
message = b"HELLO"
key = generate_key(len(message))

ciphertext = otp_encrypt(message, key)
plaintext = otp_decrypt(ciphertext, key)

print("Message   :", message)
print("Key       :", key)
print("Ciphertext:", ciphertext)
print("Decrypted :", plaintext)

# -------- Example 2: Another Message (NEW KEY!) --------
message2 = b"SECRET"
key2 = generate_key(len(message2))

cipher2 = otp_encrypt(message2, key2)
plain2 = otp_decrypt(cipher2, key2)

print("\nMessage   :", message2)
print("Key       :", key2)
print("Ciphertext:", cipher2)
print("Decrypted :", plain2)

# -------- Example 3: File-style Bytes --------
data = b"\x10\x20\x30\x40\x50"
key3 = generate_key(len(data))

cipher3 = otp_encrypt(data, key3)
plain3 = otp_decrypt(cipher3, key3)

print("\nRaw Data  :", data)
print("Ciphertext:", cipher3)
print("Decrypted :", plain3)


Message   : b'HELLO'
Key       : b'\x00\xc7\xbeG\xff'
Ciphertext: b'H\x82\xf2\x0b\xb0'
Decrypted : b'HELLO'

Message   : b'SECRET'
Key       : b'7\x86 $w\xdf'
Ciphertext: b'd\xc3cv2\x8b'
Decrypted : b'SECRET'

Raw Data  : b'\x10 0@P'
Ciphertext: b'\x8d\xaa}\x84\x8a'
Decrypted : b'\x10 0@P'
